In [26]:
import json
import pandas as pd
from crossref.restful import Works

works = Works()


# import as dataframe
df_2018 = pd.read_csv('2018_data/2018_data.csv')
df_2019 = pd.read_csv('2019_data/2019_data.csv')

# combine
df = pd.concat([df_2018, df_2019])
df['IDENTIFIER: Reference ID'] = df['REFERENCE: tag'].astype('category').cat.codes.apply(lambda x: x+1)
df.to_csv('combined_data.csv')
df = df.drop_duplicates('IDENTIFIER: Reference ID')
df

,REFERENCE: tag,REFERENCE: doi,FORMULA,PROPERTY: Type of phases,PROPERTY: Single/Multiphase,PROPERTY: synthesis method,PROPERTY: grain size ($\mu$m),PROPERTY: ROM Density (g/cm$^3$),PROPERTY: HV,PROPERTY: Type of test,...,PROPERTY: Young modulus (GPa),PROPERTY: O content (wppm),PROPERTY: N content (wppm),PROPERTY: C content (wppm),REFERENCE: comment,"Internal Reference # (J4, J90)","J4, J90 Original DOI",PROPERTY: Exp. Density (g/cm$^3$),Unnamed: 22,IDENTIFIER: Reference ID
0,4,10.1016/j.jmmm.2014.07.023,Al0.25CoFeNi,FCC,S,AC,NaN,7.9,138,C,...,196,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,27
9,5,10.1016/j.actamat.2014.08.026,CoCrFeNi,FCC,S,CR,24,8.2,NaN,T,...,225,NaN,NaN,NaN,elongation is total,NaN,NaN,NaN,NaN,37
16,6,10.1016/j.intermet.2015.01.004,CoCrFeNb0.103Ni,FCC+Laves (C14),M,AC,150-200,8.2,NaN,T,...,NaN,NaN,NaN,NaN,all YS recorded as +1 MPa,NaN,NaN,NaN,NaN,46
22,7,10.1016/j.jallcom.2009.08.090,Al0.25CoCrFeNi,FCC,S,A,NaN,7.7,110,NaN,...,216,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,54
42,8,10.1007/s11837-013-0753-6,Al0.5CoCrFeMo0.5Ni,FCC+Sigma,M,AC,NaN,7.7,425,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,63
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
941,J131,10.3390/e21010015,HfMoNbTaTiZr,BCC,S,AC,NaN,NaN,NaN,C,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,129
967,J132,10.3390/coatings9010016,CoCrFeNi,FCC,S,VHP,0.0276,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,130
970,J133,10.5755/j01.ms.25.3.19452,Al0.4FeCrCo1.5NiTi0.3,FCC + BCC,M,SPS,NaN,NaN,515 $\pm$ 16,C,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,131
971,J134,10.1007/s11661-018-4646-8,HfNbTaTiZr,BCC,S,AC,NaN,NaN,NaN,T,...,91.5,NaN,NaN,NaN,Hot Isostatically Pressed,NaN,NaN,NaN,NaN,132


In [22]:
# query crossref
ref_data = {row['IDENTIFIER: Reference ID']: works.doi(row['REFERENCE: doi']) for index, row in df.iterrows()}
json.dump(ref_data, open('reference_data.json', 'w'))

In [23]:
# add refs not in cr
ref_data[48] = {'title':['Solution strengthening of ductile refractory HfMoxNbTaTiZr high-entropy alloys'],
                 'URL':'https://doi.org/10.1016/j.matlet.2016.03.133', 
                 'issued':{'date-parts':[[2016]]}} 

In [24]:

df['REFERENCE: year'] = df['IDENTIFIER: Reference ID'].apply(lambda x: str(ref_data[x]['issued']['date-parts'][0][0]) if ref_data[x] != None else None)
df['REFERENCE: title'] = df['IDENTIFIER: Reference ID'].apply(lambda x: ref_data[x]['title'][0] if ref_data[x] != None else None)
df['REFERENCE: url'] = df['IDENTIFIER: Reference ID'].apply(lambda x: ref_data[x]['URL'] if ref_data[x] != None else None)
df

,REFERENCE: tag,REFERENCE: doi,FORMULA,PROPERTY: Type of phases,PROPERTY: Single/Multiphase,PROPERTY: synthesis method,PROPERTY: grain size ($\mu$m),PROPERTY: ROM Density (g/cm$^3$),PROPERTY: HV,PROPERTY: Type of test,...,PROPERTY: C content (wppm),REFERENCE: comment,"Internal Reference # (J4, J90)","J4, J90 Original DOI",PROPERTY: Exp. Density (g/cm$^3$),Unnamed: 22,IDENTIFIER: Reference ID,REFERENCE: year,REFERENCE: title,REFERENCE: url
0,4,10.1016/j.jmmm.2014.07.023,Al0.25CoFeNi,FCC,S,AC,NaN,7.9,138,C,...,NaN,NaN,NaN,NaN,NaN,NaN,27,2014,Effects of Al and Si addition on the structure...,http://dx.doi.org/10.1016/j.jmmm.2014.07.023
9,5,10.1016/j.actamat.2014.08.026,CoCrFeNi,FCC,S,CR,24,8.2,NaN,T,...,NaN,elongation is total,NaN,NaN,NaN,NaN,37,2014,Temperature dependence of the mechanical prope...,http://dx.doi.org/10.1016/j.actamat.2014.08.026
16,6,10.1016/j.intermet.2015.01.004,CoCrFeNb0.103Ni,FCC+Laves (C14),M,AC,150-200,8.2,NaN,T,...,NaN,all YS recorded as +1 MPa,NaN,NaN,NaN,NaN,46,2015,Effects of Nb additions on the microstructure ...,http://dx.doi.org/10.1016/j.intermet.2015.01.004
22,7,10.1016/j.jallcom.2009.08.090,Al0.25CoCrFeNi,FCC,S,A,NaN,7.7,110,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,54,2009,Microstructure and mechanical property of as-c...,http://dx.doi.org/10.1016/j.jallcom.2009.08.090
42,8,10.1007/s11837-013-0753-6,Al0.5CoCrFeMo0.5Ni,FCC+Sigma,M,AC,NaN,7.7,425,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,63,2013,Effect of Aluminum Content on Microstructure a...,http://dx.doi.org/10.1007/s11837-013-0753-6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
941,J131,10.3390/e21010015,HfMoNbTaTiZr,BCC,S,AC,NaN,NaN,NaN,C,...,NaN,NaN,NaN,NaN,NaN,NaN,129,2018,"Effects of Mo, Nb, Ta, Ti, and Zr on Mechanica...",http://dx.doi.org/10.3390/e21010015
967,J132,10.3390/coatings9010016,CoCrFeNi,FCC,S,VHP,0.0276,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,130,2018,Microstructure and Properties of CoCrFeNi(WC) ...,http://dx.doi.org/10.3390/coatings9010016
970,J133,10.5755/j01.ms.25.3.19452,Al0.4FeCrCo1.5NiTi0.3,FCC + BCC,M,SPS,NaN,NaN,515 $\pm$ 16,C,...,NaN,NaN,NaN,NaN,NaN,NaN,131,2019,Enhancement of Mechanical Properties and Corro...,http://dx.doi.org/10.5755/j01.ms.25.3.19452
971,J134,10.1007/s11661-018-4646-8,HfNbTaTiZr,BCC,S,AC,NaN,NaN,NaN,T,...,NaN,Hot Isostatically Pressed,NaN,NaN,NaN,NaN,132,2018,Effect of Cold Deformation and Annealing on th...,http://dx.doi.org/10.1007/s11661-018-4646-8


In [25]:
cols = ['IDENTIFIER: Reference ID', 'REFERENCE: title', 'REFERENCE: year', 'REFERENCE: doi', 'REFERENCE: url']
df = df[cols]

df.sort_values('IDENTIFIER: Reference ID').to_csv('references.csv', index=False)